In [65]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import my_lib
import csv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
pipeline = my_lib.Pipeline()
raw_df = pipeline.load_data()
raw_df = pipeline.encode_to_category(raw_df)
raw_df = pipeline.tokenize(raw_df)
raw_df.head()

,tweet,label,enterprise,is_irrelevant,token_list,clean_tweet,number_vector,input_vector
0,20 min line @apple store @short pump.,1,0,0,20 min line @apple store @short pump.,20 min line @apple store @short pump .,"[414, 669, 408, 115, 544, 52, 408, 202, 544, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Nueva tecnología convierte cualquier superfici...,3,1,1,nueva tecnología convierte cualquier superfici...,nueva tecnología convierte cualquier superfici...,"[52, 443, 198, 127, 419, 408, 205, 198, 172, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Some people should not post replies in #Google...,1,2,0,some people should not post replies in #google...,some people should not post replies in #google...,"[368, 110, 115, 198, 408, 602, 198, 110, 602, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,I know a few others having same issue RT @Joel...,2,0,0,i know a few others having same issue rt @joel...,i know a few others having same issue rt @joel...,"[544, 408, 502, 52, 110, 547, 408, 419, 408, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"#Microsoft - We put the """"backwards"""" into bac...",2,1,0,"#microsoft - we put the """"backwards"""" into bac...","#microsoft - we put the "" "" backwards "" "" ...","[671, 115, 544, 172, 312, 110, 368, 110, 651, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [67]:
# df = raw_df.loc[raw_df.label.isin([0,2])]
df = raw_df.loc[raw_df.label.isin([0,1,2,3])]
df = my_lib.processing.remove_useless_col(df, keep_col_list=["label", "clean_tweet"])
df_train_data, df_test_data = my_lib.processing.split_data(df)
print(f"SHAPE -> train : {df_train_data.shape}, test : {df_test_data.shape}")
df.head()

SHAPE -> train : (3338, 2), test : (835, 2)


,label,clean_tweet
0,1,20 min line @apple store @short pump .
1,3,nueva tecnología convierte cualquier superfici...
2,1,some people should not post replies in #google...
3,2,i know a few others having same issue rt @joel...
4,2,"#microsoft - we put the "" "" backwards "" "" ..."


In [68]:
for label_id, label_name in enumerate(["pos", "neu", "neg", "irr"]):
    tmp_df = df.loc[df["label"]==label_id, ["clean_tweet"]]
    for idx, row in tmp_df.iterrows():
        row.to_csv(f"/data/tensorflow_bert_data/{label_name}/text_{idx}.txt", sep=" ", index=None,
                quotechar=" ", header=False)

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import numpy as np

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", num_labels=4)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_265', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data = tf.keras.preprocessing.text_dataset_from_directory('/data/tensorflow_bert_data/', seed=123, batch_size=30000)
data

In [ ]:
for i in data.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

df_data = pd.DataFrame([train_feat, train_lab]).T
df_data.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df_data['DATA_COLUMN'] = df_data['DATA_COLUMN'].str.decode("utf-8")
print(df_data.shape)
df_data.head()

In [ ]:
row_number = len(df_data)
print(row_number)
df_train = pd.DataFrame()
df_test = pd.DataFrame()

for i in range(3):
    tmp_df = df_data.loc[df_data["LABEL_COLUMN"]==i]

    msk = np.random.rand(len(tmp_df)) < 0.8
    df_train = pd.concat([df_train, tmp_df[msk]])
    df_test = pd.concat([df_test, tmp_df[~msk]])

df_train.shape, df_test.shape

In [ ]:
df_train[["DATA_COLUMN", "LABEL_COLUMN"]].groupby("LABEL_COLUMN").count()

In [ ]:
df_test[["DATA_COLUMN", "LABEL_COLUMN"]].groupby("LABEL_COLUMN").count()



In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN],
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN],
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(df_train,
                                                                           df_test,
                                                                           'DATA_COLUMN',
                                                                           'LABEL_COLUMN')

In [ ]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(df_train, df_test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                  'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie']